# Partie 1

## Modules

In [27]:
import openpyxl
from read_input import read_input

from gurobipy import *
import math
import numpy 
import time

## Lecture des données

In [28]:
start_time = time.time()

ville="Poland" #  changer 
ressources,ressources_unavailabilities,tasks,tasks_unavailabilities = read_input(2,ville)
# V1 : "Bordeaux", "Finland", "GuineaGolf", "Italy", "Poland"
# V2 : "Austria", "Australia", "Bordeaux", "Poland", "Spain"

## Structures de données

In [29]:
def dist(lat1, long1, lat2, long2):
    """
    fonction renvoyant la distance en kms entre deux points décrits par leur latitude et longitude
    """
    if lat1==lat2 and long1==long2:
        return 0
    return(numpy.arccos(numpy.sin(2*numpy.pi/360*lat1)*numpy.sin(2*numpy.pi/360*lat2)+numpy.cos(2*numpy.pi/360*lat1)*numpy.cos(2*numpy.pi/360*lat2)*numpy.cos(2*numpy.pi/360*(long2-long1)))*6371)

#ACOS(SIN(lat1)*SIN(lat2)+COS(lat1)*COS(lat2)*COS(lon2-lon1))*6371
vitesse=5/6

L_ressources = [j for j in ressources] # tableau des objets ressources 
L_tasks = [i for i in tasks] # tableau des objets taches
number_of_ressources = len(L_ressources)
number_of_tasks = len(L_tasks)
dict_pos = {task : (task.latitude, task.longitude) for task in tasks.values()}

dict_pos = {task : (tasks[L_tasks[task]].latitude, tasks[L_tasks[task]].longitude) for task in range(number_of_tasks)}
print(dict_pos)


runav_id = []
for j in range(len(L_ressources)):
    for u in range(len(ressources[L_ressources[j]].unavailabilities)):
        runav_id.append((j,u))
number_of_runavailabilities = len(runav_id)

tunav_id = []
for i in range(len(L_tasks)):
    for v in range(len(tasks[L_tasks[i]].unavailabilities)):
        tunav_id.append((i,v))
number_of_tunavailabilities = len(tunav_id)

#distance[a][b] donne la distance de la tâche a vers la tâche b
def distance_tache_unav():
    distances = [[0 for j in range(number_of_tasks+number_of_runavailabilities)] for i in range(number_of_tasks+number_of_runavailabilities)]
    for b in range(number_of_tasks+number_of_runavailabilities) :
        for a in range(number_of_tasks+number_of_runavailabilities) :
            if a >= number_of_tasks and b >= number_of_tasks :
                distances[a][b] = dist(ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].longitude, ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].longitude)
            elif a >= number_of_tasks and b < number_of_tasks :
                distances[a][b] = dist(ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude)
            elif a < number_of_tasks and b >= number_of_tasks :
                distances[a][b] = dist(ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].longitude, tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude)
            else :
                distances[a][b] = dist(tasks[L_tasks[a]].latitude,tasks[L_tasks[a]].longitude, tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude)
    return distances
distances=distance_tache_unav()

#distance_m[j][i] donne la distance de la maison de j la tâche i
def distance_maison_unav():
    distances_m = [[0 for j in range(number_of_tasks+number_of_runavailabilities)] for i in range(number_of_ressources)]
    for t in range(number_of_tasks+number_of_runavailabilities) :
        for r in range(number_of_ressources) :
            if t >= number_of_tasks :
                distances_m[r][t] = dist(ressources[L_ressources[runav_id[t-number_of_tasks][0]]].unavailabilities[runav_id[t-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[t-number_of_tasks][0]]].unavailabilities[runav_id[t-number_of_tasks][1]].longitude, ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude)
            else :
                distances_m[r][t] = dist(ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude, tasks[L_tasks[t]].latitude, tasks[L_tasks[t]].longitude)
    return distances_m
distances_m=distance_maison_unav()


#distance_m[j][i] donne la distance de la maison de j la tâche i
#distances_m = [[dist(ressources[L_ressources[j]].latitude,ressources[L_ressources[j]].longitude, tasks[L_tasks[i]].latitude, tasks[L_tasks[i]].longitude) for i in range(number_of_tasks)] for j in range(number_of_ressources)]
#print(distances_m)

# https://gurobi.github.io/modeling-examples/technician_routing_scheduling/technician_routing_scheduling.html

{0: (54.197184, 21.930926), 1: (54.1666488, 21.8967233), 2: (53.264781, 20.737952), 3: (53.2746464, 20.9102882), 4: (54.0079614, 22.0121995), 5: (53.331325, 20.7060784), 6: (53.9905791, 21.9207052), 7: (54.1664042, 21.9604517), 8: (53.0348725, 20.7724214), 9: (54.100773, 22.2264029), 10: (53.2914688, 20.7159019), 11: (53.0322502, 21.0491345), 12: (53.3124742, 20.7560638), 13: (54.0818592, 21.9349896), 14: (54.1752805, 21.8810835), 15: (54.2129531, 22.0596256), 16: (53.3225801, 20.7288242), 17: (53.2970184, 20.7543235), 18: (53.2853981, 20.7468738)}


## Variables de décision

In [30]:
### Create Model
m = Model("trs0")

M = 1440

print(number_of_ressources)
print(number_of_tasks)

# x[(number_of_tasks+u,r)]
# runav_id[u (general)]=(r,id (pour la personne))
### Decision variables
# Task-ressource assignment
x = {(t,r) : m.addVar(vtype = GRB.BINARY, name = f'x_{t}_{r}') for r in range(number_of_ressources) for t in range(number_of_tasks)}
for r in range(number_of_ressources):
    for u in range(number_of_runavailabilities):
        if runav_id[u][0]==r:
            x[(number_of_tasks+u,r)]=1
        else:
            x[(number_of_tasks+u,r)]=0

# Time-task assignment
s = {t : m.addVar(vtype = GRB.INTEGER, lb = 0, ub = 1440, name = f's_{t}') for t in range(number_of_tasks)}
#Les s_t valent entre 0 et 1440 car les contraintes de temps (entre 8h et 18h) sont contenues dans les jeux de données.
for u in range(number_of_runavailabilities):
    s[number_of_tasks+u]=ressources[L_ressources[runav_id[u][0]]].unavailabilities[runav_id[u][1]].unavailabilityStart

# Est-ce que le travailleur se déplace entre les points a et b ?
y = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'y_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities)}# if a != b}
# Technician cannot leave or return to a depot that is not its base

# Is the worker used ?
u = {r: m.addVar(vtype = GRB.BINARY, name = f'u_{r}') for r in range(number_of_ressources)}

# starting task of worker
d = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'd_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks+number_of_runavailabilities)}
# ending task of worker
f = {(r,t) : m.addVar(vtype = GRB.BINARY, name = f'f_{r}_{t}') for r in range(number_of_ressources) for t in range(number_of_tasks+number_of_runavailabilities)}

#variables de linéarisation
#H = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'h_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}
#P = {(r,a,b) : m.addVar(vtype = GRB.BINARY,lb = 0, ub = 1440, name = f'p_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks) for b in range(number_of_tasks)}# if a != b}

a = {v : m.addVar(vtype = GRB.BINARY, name = f'a_{v}') for v in range(number_of_tunavailabilities)}

#variables de pause midi
z = {(r,a,b) : m.addVar(vtype = GRB.BINARY, name = f'z_{r}_{a}_{b}') for r in range(number_of_ressources) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities)}# if a != b}
zb = {(r) : m.addVar(vtype = GRB.BINARY, name = f'z_{r}') for r in range(number_of_ressources)}
ze = {(r) : m.addVar(vtype = GRB.BINARY, name = f'z_{r}') for r in range(number_of_ressources)}



2
19


## Contraintes

In [31]:
# une personne est obligée de se déplacer (pas d'arrêtes de bouclage sur un point du graphe)
C4_2 = [m.addConstr(y[(r,a,a)] == 0) for r in range(number_of_ressources) for a in range(number_of_tasks+number_of_runavailabilities)]

In [32]:
nbRpT = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpT[t]+=x[(t,r)]

# One ressource per task
C1 = [m.addConstr(nbRpT[t] <= 1) for t in range(number_of_tasks)]

In [33]:
nbRpTs = [LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
     nbRpTs[t]+=x[(t,r)]*(ressources[L_ressources[r]].skill!=tasks[L_tasks[r]].skill)

 # Skilled technicians
C2 = [m.addConstr(nbRpTs[t] == 0) for t in range(number_of_tasks)]

In [34]:
nbRpTl =[ LinExpr() for t in range(number_of_tasks)]
for t in range(number_of_tasks):
  for r in  range(number_of_ressources):
    nbRpTl[t]+=x[(t,r)]*(ressources[L_ressources[r]].level-tasks[L_tasks[t]].level)

# Leveled technicians
C3 = [m.addConstr(nbRpTl[t] >= 0) for t in range(number_of_tasks)]

In [35]:
# si une personne est affectée, elle arrive jusqu'à la tâche
C4 = {(i,j) : m.addConstr(quicksum(y[(j,a,i)] for a in range(number_of_tasks+number_of_runavailabilities)) + d[(j,i)] == x[(i,j)], name = f'arrive{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks+number_of_runavailabilities)}

# une personne doit partir après avoir exécuté sa tâche
C5 = {(i,j) : m.addConstr(quicksum(y[(j,i,b)] for b in range(number_of_tasks+number_of_runavailabilities)) + f[(j,i)] == x[(i,j)], name = f'part{i}{j}') for j in range(number_of_ressources) for i in range(number_of_tasks+number_of_runavailabilities)}

In [36]:
# non overlapping tasks
#print(vitesse)
#print(tasks[L_tasks[0]].duration)
C6 = [[m.addConstr(s[b] >= s[a] + tasks[L_tasks[a]].duration + distances[a][b]/vitesse -M*(1-quicksum(y[(j,a,b)] for j in range(number_of_ressources)))) for b in range(number_of_tasks+number_of_runavailabilities)] for a in range(number_of_tasks)]
C6bis = [[m.addConstr(s[b] >= ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd + distances[a][b]/vitesse -M*(1-quicksum(y[(j,a,b)] for j in range(number_of_ressources)))) for b in range(number_of_tasks+number_of_runavailabilities)] for a in range(number_of_tasks,number_of_tasks+number_of_runavailabilities)]
C6bisbis = [[m.addConstr(ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].unavailabilityStart >= s[a] + tasks[L_tasks[a]].duration + distances[a][b]/vitesse -M*(1-quicksum(y[(j,a,b)] for j in range(number_of_ressources)))) for b in range(number_of_tasks, number_of_tasks+number_of_runavailabilities)] for a in range(number_of_tasks)]

In [37]:
# une personne doit partir de chez elle si elle travaille ce jour la
C7 = [m.addConstr(quicksum(d[(j,a)] for a in range(number_of_tasks+number_of_runavailabilities)) == u[j]) for j in range(number_of_ressources)]
# une personne doit revenir chez elle si elle travaille ce jour la
C8 = [m.addConstr(quicksum(f[(j,b)] for b in range(number_of_tasks+number_of_runavailabilities)) == u[j]) for j in range(number_of_ressources)]

# contraintes pour lier x et u
C9 = [m.addConstr(quicksum(x[(i,j)] for i in range(number_of_tasks+number_of_runavailabilities)) <= u[j]*M) for j in range(number_of_ressources)]

In [38]:
# la premiere tache commence apres le début de la journée
#M=10E2

# LES INDISPOS SONT EXCLUES
C10 = [m.addConstr(s[i] >= ressources[L_ressources[j]].workingStart -M*(1-x[(i,j)]) + distances_m[j][i]/vitesse) for i in range(number_of_tasks) for j in range(number_of_ressources)]

In [39]:
# la derniere tache finit avant la fin de la journee
C11 = [m.addConstr(ressources[L_ressources[j]].workingEnd >= s[i]+tasks[L_tasks[i]].duration + distances_m[j][i]/vitesse -M*(1-x[(i,j)])) for i in range(number_of_tasks) for j in range(number_of_ressources)]
# LES INDISPOS SONT EXCLUES

#C11bis = [m.addConstr(
#    ressources[L_ressources[j]].workingEnd >= 
#    ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd + 
#    distances_m[j][i]/vitesse -M*(1-x[(i,j)]) )
#    for i in range(number_of_tasks,number_of_tasks+number_of_runavailabilities) for j in range(number_of_ressources) ]



In [40]:
C14 = [m.addConstr(s[t] -M*(1-quicksum(x[(t,j)] for j in range (number_of_ressources))) + tasks[L_tasks[t]].duration - M*a[i] <= tasks[L_tasks[t]].unavailabilities[u].unavailabilityStart) for i,(t,u) in enumerate(tunav_id)]
C15 = [m.addConstr(s[t] -M*(1-quicksum(x[(t,j)] for j in range (number_of_ressources))) + M*(1-a[i]) >= tasks[L_tasks[t]].unavailabilities[u].unavailabilityEnd) for i,(t,u) in enumerate(tunav_id)]



In [41]:
# for r in range(number_of_ressources):
#   for b in range (number_of_tasks+number_of_runavailabilities):
#     for a in range(number_of_tasks,number_of_tasks+number_of_runavailabilities):
#       try:
#         m.addConstr(60+ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd <= -distances[a][b]/vitesse+ s[b] + z[(r,a,b)]*M )
#       except:
#         print(r,b,a)

In [42]:
Cz1 = [[m.addConstr(z[(r,a,b)]<=y[(r,a,b)]) for r in range(number_of_ressources)] for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities)]
Cz2 = [m.addConstr(quicksum(z[(r,a,b)] for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities))+zb[r]+ze[r]==u[r]) for r in range(number_of_ressources)]

In [43]:
Cz3 = [m.addConstrs(M*(1-x[a,r])+s[a]+M*(1-zb[r]) >= 13*60 for a in range(number_of_tasks+number_of_runavailabilities) for r in range(number_of_ressources))] 

Cz4 = [m.addConstrs(s[a] + tasks[L_tasks[a]].duration <= 13*60 + M*(1-x[a,r]) + M*(1-ze[r]) for r in range(number_of_ressources) for a in range(number_of_tasks))]
Cz4bis = [m.addConstrs(ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd <= 13*60 + M*(1-x[a,r]) +M*(1-zb[r]) for r in range(number_of_ressources) for a in range(number_of_tasks,number_of_tasks+number_of_runavailabilities))]


In [44]:
Cz5 = [[[m.addConstr(distances[a][b]/vitesse+s[a]+60+tasks[L_tasks[a]].duration <= s[b] + (1-z[(r,a,b)])*M) for r in range(number_of_ressources)] for b in range(number_of_tasks+number_of_runavailabilities)] for a in range(number_of_tasks)]
Cz5bis = [[[m.addConstr(60+ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd <= -distances[a][b]/vitesse+ s[b] + (1-z[(r,a,b)])*M ) for r in range(number_of_ressources)] for b in range (number_of_tasks+number_of_runavailabilities)] for a in range(number_of_tasks,number_of_tasks+number_of_runavailabilities)]


Cz6 = [m.addConstrs(s[b] >= 13*60 - M*(1-quicksum(z[(r,a,b)] for r in range(number_of_ressources))) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities))] 

Cz7 = [m.addConstrs(s[a] + tasks[L_tasks[a]].duration <= 13*60 + M*(1-quicksum(z[(r,a,b)] for r in range(number_of_ressources))) for a in range(number_of_tasks) for b in range(number_of_tasks+number_of_runavailabilities))]
Cz7bis = [m.addConstrs(ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd <= 13*60 + M*(1-quicksum(z[(r,a,b)] for r in range(number_of_ressources))) for a in range(number_of_tasks,number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks, number_of_tasks+number_of_runavailabilities))]


In [45]:
# contrainte évitant d'avoir des cycles dans les graphes (permet que les tâches soient reliées entre elles)
#C19 = [m.addConstr(y[(j,a,b)] + y[(j,b,a)] <=1) for a in range(number_of_tasks+number_of_runavailabilities) for b in range(number_of_tasks+number_of_runavailabilities) for j in range(number_of_ressources)]
#C20 = [m.addConstr(d[(j,a)] + f[(j,a)] <=1) for a in range(number_of_tasks+number_of_runavailabilities) for j in range(number_of_ressources)]

## Fonction objectif et résolution

In [46]:
epsilon = 0.0001

couts=LinExpr()
ctot=0
for j in range(number_of_ressources):
  for a in range(number_of_tasks+number_of_runavailabilities):
    for b in range(number_of_tasks+number_of_runavailabilities):
      couts+=y[(j,a,b)]*distances[a][b]/vitesse
      if j==0:
        ctot+=distances[a][b]/vitesse
    couts+=d[(j,a)]*distances_m[j][a]/vitesse
    couts+=f[(j,a)]*distances_m[j][a]/vitesse

duree=LinExpr()
dtot=0
for t in range(number_of_tasks):
  for r in range(number_of_ressources):
    duree+=x[(t,r)]*tasks[L_tasks[t]].duration
  dtot+=tasks[L_tasks[t]].duration


# minimiser-(la durée totale des tâches exécutées)
obj1 = -duree/dtot # normalisation
# minimiser le coût opérationnel
obj2 = couts/ctot # normalisation

obj = 0.8*obj1+0.2*obj2

solutions_efficaces = list()

# -- Ajout de la fonction objectif --
m.setObjective(obj, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet
m.update()
m.display()
m.optimize()


# it = 0

if  m.status == GRB.INFEASIBLE:
  print("il n'y a pas de solution !!")

# while m.status != GRB.INFEASIBLE:
#     sol = (round(m.objVal,2), round(obj2.getValue()*100,2))
#     print("itération {} solution {}".format(it, sol))
#     solutions_efficaces.append(sol)
    
    
#     # - 
#     it += 1
#     # -- Ajout de l'epsilon constraint
#     m.addConstr(obj2 <= obj2.getValue() - epsilon, name=f'epsilon_constraint_{it}')
#     # -- Mise à jour du modèle  --
#     m.update()
#     # -- Résolution --
#     m.optimize()

In [ ]:
"""
solve with epsilon
epsilon = 0.0001

couts=LinExpr()
for j in range(number_of_ressources):
  for a in range(number_of_tasks+number_of_runavailabilities):
    for b in range(number_of_tasks+number_of_runavailabilities):
      couts+=y[(j,a,b)]*distances[a][b]/vitesse
    couts+=d[(j,a)]*distances_m[j][a]/vitesse
    couts+=f[(j,a)]*distances_m[j][a]/vitesse

duree=LinExpr()
for r in range(number_of_ressources):
  for t in range(number_of_tasks):
    duree+=x[(t,r)]*tasks[L_tasks[t]].duration

# minimiser-(la durée totale des tâches exécutées)
obj1 = -duree
# minimiser le coût opérationnel
obj2 = couts

solutions_efficaces = list()

# -- Ajout de la fonction objectif --
m.setObjective(obj1, GRB.MINIMIZE)

# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet
m.update()
m.display()
m.optimize()


it = 0

if  m.status == GRB.INFEASIBLE:
  print("il n'y a pas de solution !!")

while m.status != GRB.INFEASIBLE:
    sol = (round(m.objVal,2), round(obj2.getValue()*100,2))
    print("itération {} solution {}".format(it, sol))
    solutions_efficaces.append(sol)

    if it ==20:
      break
    
    
    # - 
    it += 1
    # -- Ajout de l'epsilon constraint
    m.addConstr(obj2 <= obj2.getValue() - epsilon, name=f'epsilon_constraint_{it}')
    # -- Mise à jour du modèle  --
    m.update()
    # -- Résolution --
    m.optimize()
"""


'\nsolve with epsilon\nepsilon = 0.0001\n\ncouts=LinExpr()\nfor j in range(number_of_ressources):\n  for a in range(number_of_tasks+number_of_runavailabilities):\n    for b in range(number_of_tasks+number_of_runavailabilities):\n      couts+=y[(j,a,b)]*distances[a][b]/vitesse\n    couts+=d[(j,a)]*distances_m[j][a]/vitesse\n    couts+=f[(j,a)]*distances_m[j][a]/vitesse\n\nduree=LinExpr()\nfor r in range(number_of_ressources):\n  for t in range(number_of_tasks):\n    duree+=x[(t,r)]*tasks[L_tasks[t]].duration\n\n# minimiser-(la durée totale des tâches exécutées)\nobj1 = -duree\n# minimiser le coût opérationnel\nobj2 = couts\n\nsolutions_efficaces = list()\n\n# -- Ajout de la fonction objectif --\nm.setObjective(obj1, GRB.MINIMIZE)\n\n# -- Choix d\'un paramétrage d\'affichage minimaliste --\nm.params.outputflag = 0 # mode muet\nm.update()\nm.display()\nm.optimize()\n\n\nit = 0\n\nif  m.status == GRB.INFEASIBLE:\n  print("il n\'y a pas de solution !!")\n\nwhile m.status != GRB.INFEASIBLE:\

In [ ]:
# -- Affichage de la solution --

t=open("Solution"+ville+"V2ByV4.txt",'w'.format(ville))
res="taskId;performed;employeeName;startTime;\n"
for j in range(number_of_tasks):
  p=False
  for i in range(number_of_ressources):
    if round(x[(j,i)].x):
      res+="{};{};{};{};\n".format(L_tasks[j],1,L_ressources[i],int(s[j].x))
      p=True
  if not p:
    res+="{};0;;;\n".format(L_tasks[j])
#print(res)
t.write(res)
t.close()

startTimesT = [s[t].x for t in range(number_of_tasks)]
startTimesU = [ressources[L_ressources[runav_id[t][0]]].unavailabilities[runav_id[t][1]].unavailabilityStart for t in range(number_of_runavailabilities)]
endTimesT = [s[t].x+tasks[L_tasks[t]].duration for t in range(number_of_tasks)]
endTimesU = [ressources[L_ressources[runav_id[t][0]]].unavailabilities[runav_id[t][1]].unavailabilityEnd for t in range(number_of_runavailabilities)]
startTimes=startTimesT+startTimesU
endTimes=endTimesT+endTimesU

for i in range(number_of_ressources):
  print("Employé(e) : {}".format(L_ressources[i]))
  for j in range(number_of_tasks):
    if  round(d[(i,j)].x)==1:
      print(" --> Déplacement de la maison vers {} ({} minutes)".format(L_tasks[j],round(distances_m[i][j]/vitesse)))
  for j in range(number_of_runavailabilities):
    if  round(d[(i,j+number_of_tasks)].x)==1:
      print(" --> Déplacement de la maison vers indispo ({} minutes)".format(round(distances_m[i][j+number_of_tasks]/vitesse)))
  for j in sorted(range(number_of_tasks+number_of_runavailabilities),key=lambda u: int(startTimes[u])):
    if (j>=number_of_tasks and runav_id[j-number_of_tasks][0]==i) or (j<number_of_tasks and round(x[(j,i)].x)):
      td=round(startTimes[j])
      tf=round(endTimes[j])
      if j<number_of_tasks:
        print(" - Tâche {} de {}h{} à {}h{}".format(L_tasks[j],td//60,td%60,tf//60,tf%60))
        for b in range(number_of_tasks):
          if  round(y[(i,j,b)].x)==1:
            print(" --> Déplacement de {} vers {} ({} minutes)".format(L_tasks[j],L_tasks[b],round(distances[j][b]/vitesse)))
        for b in range(number_of_runavailabilities):
          if  round(y[(i,j,b+number_of_tasks)].x)==1:
            print(" --> Déplacement de {} vers indispo ({} minutes)".format(L_tasks[j],round(distances[j][b+number_of_tasks]/vitesse)))
      else:
        print(" - Indispo de {}h{} à {}h{}".format(td//60,td%60,tf//60,tf%60))
        for b in range(number_of_tasks):
          if  round(y[(i,j,b)].x)==1:
            print(" --> Déplacement de indispo vers {} ({} minutes)".format(L_tasks[b],round(distances[j][b]/vitesse)))
        for b in range(number_of_runavailabilities):
          if  round(y[(i,j,b+number_of_tasks)].x)==1:
            print(" --> Déplacement de indispo vers indispo ({} minutes)".format(round(distances[j][b+number_of_tasks]/vitesse)))
      
  for j in range(number_of_tasks):
    if  round(f[(i,j)].x)==1:
      print(" --> Déplacement de {} vers la maison ({} minutes)".format(L_tasks[j],round(distances_m[i][j]/vitesse)))
  for j in range(number_of_runavailabilities):
    if  round(f[(i,j+number_of_tasks)].x)==1:
      print(" --> Déplacement de indispo vers la maison ({} minutes)".format(round(distances_m[i][j+number_of_tasks]/vitesse)))

interval = time.time() - start_time

Employé(e) : Irena
 --> Déplacement de la maison vers indispo (0 minutes)
 - Indispo de 10h0 à 11h15
 --> Déplacement de indispo vers la maison (0 minutes)
Employé(e) : Karol


In [ ]:
print("temps de calcul : ", interval)
print("valeur de la fonction objectif : ", m.ObjVal)

temps de calcul :  6.68965744972229
valeur de la fonction objectif :  0.0


## Vérification de la solution

In [ ]:
# affichage de la carte
import folium

lat_tot,long_tot=0,0
for a in range(number_of_tasks):
  lat_tot+=tasks[L_tasks[a]].latitude
  long_tot+=tasks[L_tasks[a]].longitude
for a in range(number_of_tasks, number_of_tasks+number_of_runavailabilities):
  lat_tot+=ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].latitude
  long_tot+=ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].longitude
mid_latitude = lat_tot/(number_of_tasks+number_of_runavailabilities)
mid_longitude = long_tot/(number_of_tasks+number_of_runavailabilities)
carte=folium.Map(location=[mid_latitude, mid_longitude],zoom_start=10)
for a in range(number_of_tasks):
  folium.Marker([tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude],icon=folium.Icon(color='black')).add_to(carte)
for a in range(number_of_tasks,number_of_tasks+number_of_runavailabilities):
  folium.Marker([ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].latitude,
   ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].longitude],icon=folium.Icon(color='orange')).add_to(carte)
list_colors = ['darkgreen', 'blue', 'red', 'green', 'darkpurple', 'lightred', 'purple', 'pink', 'orange', 'darkblue', 'gray', 'darkred']
number_of_colors = len(list_colors)
counter = 0
for r in range(number_of_ressources):
  color_r = list_colors[counter % number_of_colors]
  folium.Marker([ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude],icon=folium.Icon(color=color_r)).add_to(carte)
  for a in range(number_of_tasks+number_of_runavailabilities):
    for b in range(number_of_tasks+number_of_runavailabilities):
      if a < number_of_tasks:
        lat_a,long_a = tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude
      else:
        lat_a,long_a = ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].longitude
      if b < number_of_tasks :
        lat_b,long_b = tasks[L_tasks[b]].latitude, tasks[L_tasks[b]].longitude
      else :
        lat_b,long_b = ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].latitude, ressources[L_ressources[runav_id[b-number_of_tasks][0]]].unavailabilities[runav_id[b-number_of_tasks][1]].longitude
      if round(y[(r,a,b)].x) == 1:
        line_ab = [(lat_a, long_a),(lat_b, long_b)]
        folium.PolyLine(line_ab, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
    if round(d[(r,a)].x) == 1:
      line_home_t = [(ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude), (tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude)]
      folium.PolyLine(line_home_t, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
    if round(f[(r,a)].x) == 1:
      line_t_home = [(tasks[L_tasks[a]].latitude, tasks[L_tasks[a]].longitude), (ressources[L_ressources[r]].latitude,ressources[L_ressources[r]].longitude)]
      folium.PolyLine(line_t_home, color=color_r, weight=2.5, opacity=0.8).add_to(carte)
  counter+=1

carte.save('Cartographie/Cartev2.html')

carte

ModuleNotFoundError: No module named 'folium'

In [ ]:
import plotly.express as px
import pandas as pd

# plotting the working tasks
liste=[]
for i in range(number_of_ressources):
    for j in range(number_of_tasks):
        if round(x[(j,i)].x):
            start=int(s[j].x)
            end=(int(s[j].x)+tasks[L_tasks[j]].duration)
            liste.append(dict(Task=L_ressources[i], Start=start, Finish=end, Resource="executing task",J="tâche "+str(j+1)))

# plotting the ressource unavailabilities
for r in range(number_of_ressources):
    for R,id in runav_id:
        if r == R:
            start= ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityStart
            end= ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd
            liste.append(dict(Task=L_ressources[r], Start=start, Finish=end, Resource="unavailable",J="unavailability "+str(id+1)))

# plotting the transports
for a in range(number_of_tasks):
    for b in range(number_of_tasks):
        for j in range(number_of_ressources):
            if round(y[(j,a,b)].x):
                start=(int(s[a].x)+tasks[L_tasks[a]].duration)
                end=(start+distances[a][b]/vitesse)
                liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="moving",J="trajet "+str(a+1)+"-"+str(b+1)))

# plotting the first transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(d[(j,a)].x):
            start=(s[a].x-distances_m[j][a]/vitesse)
            end=s[a].x
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J="trajet maison -"+str(a+1)))

# plotting the last transport
for a in range(number_of_tasks):
    for j in range(number_of_ressources):
        if round(f[(j,a)].x):
            start=(s[a].x + tasks[L_tasks[i]].duration)
            end=(start+ distances_m[j][a]/vitesse)
            liste.append(dict(Task=L_ressources[j], Start=start, Finish=end, Resource="depart",J=str(a+1)+"- trajet maison"))

df = pd.DataFrame(liste)
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task",text="J")

fig.layout.xaxis.type = 'linear'
fig.data[0].x = df.delta.tolist()
fig.show()

In [ ]:
# check que les techniciens ont le bon niveau et les bonnes compétences pour les tâches exécutées
a = True
for j in range(number_of_ressources):
    for i in range(number_of_tasks):
        if round(x[(i,j)].x):
            # check que le skill est bon
            if ressources[L_ressources[j]].skill!=tasks[L_tasks[i]].skill:
                print("fail r,t ",j,i)
                a = False
            # check que le level est bon
            if ressources[L_ressources[j]].level < tasks[L_tasks[i]].level:
                print("fail r,t ",j,i)
                a = False
if a :
    print("Les techniciens ont bien les bonnes compétences et le bon niveau pour toutes les tâches exécutées")
            

0
19


In [ ]:
# check que les contraintes d'ouverture et de fermeture des tâches sont respectées en début et fin de journée
a = True
for j in range(number_of_ressources):
    for i in range(number_of_tasks):
        if round(x[(i,j)].x):
            if s[i].x < tasks[L_tasks[i]].openingTime:
                print("fail r,t ",j,i)
                a = False
            if tasks[L_tasks[i]].closingTime < s[i].x + tasks[L_tasks[i]].duration :
                print("fail r,t ",j,i)
                a = False

            
            
if a :
    print("Les contraintes d'ouverture et de fermeture des tâches sont respectées")

Les contraintes d'ouverture et de fermeture des tâches sont respectées


In [ ]:
# chaque employé possède une pause midi d’une durée d’1h à placer entre 12h et 14h, dans un lieu quelconque
bool = False
for i in range(number_of_ressources):
    bool = False
    if round(ze[i].x):
        print("l'employé ", L_ressources[i], " fait sa pause à la fin de sa journée ")
        bool = True
    elif round(zb[i].x):
        print("l'employé ", L_ressources[i], " fait sa pause au début de sa journée ")
        bool = True
    for a in range(number_of_tasks+number_of_runavailabilities):
        for b in range(number_of_tasks+number_of_runavailabilities):
            if round(u[i].x)==0:
                bool=True
            if round(z[(i,a,b)].x) and round(u[i].x):
                bool = True
                print("l'employé ", L_ressources[i], " fait sa pause entre les tâches ", a+1, "et ", b+1)
    if bool ==False:
        print("l'employé ", L_ressources[i], "n'a pas de pause !!")

l'employé  John  fait sa pause entre les tâches  9 et  8
l'employé  Sam  fait sa pause entre les tâches  17 et  1
l'employé  David  fait sa pause à la fin de sa journée 


In [ ]:
# certains employés peuvent avoir des périodes d’indisponibilité dans la journée
 
bool = True
for r in range(number_of_ressources):
    for R,id in runav_id:
        if r == R:
            for t in range(number_of_tasks):
                if round(x[(t,r)].x):
                    if s[t].x<= ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityStart:
                        if s[t].x+ tasks[L_tasks[j]].duration > ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityStart:
                            print("la personne ",  L_ressources[r], "fait la tâche ", t+1, "alors qu'elle a une indisponibilité ", id+1)
                            bool = False
                    elif s[t].x< ressources[L_ressources[runav_id[a-number_of_tasks][0]]].unavailabilities[runav_id[a-number_of_tasks][1]].unavailabilityEnd:
                        print("la personne ",  L_ressources[r], "fait la tâche ", t+1, "alors qu'elle a une indisponibilité ", id+1)
                        bool = False

if bool :
    print("les indispos de ressource sont respectées")

# x[(number_of_tasks+u,r)]
# runav_id[u (general)]=(r,id (pour la personne))

les indispos de ressource sont respectées


In [ ]:
# certaines tâches ne peuvent pas être faites à certains créneaux
 
b = True
for t in range(number_of_tasks):
    for T,id in tunav_id:
        if t == T:
            for r in range(number_of_ressources):
                if round(x[(t,r)].x):
                    if s[t].x<= tasks[L_tasks[T]].unavailabilities[id].unavailabilityStart:
                        if s[t].x+ tasks[L_tasks[T]].duration > tasks[L_tasks[T]].unavailabilities[id].unavailabilityStart:
                            print("la personne ", r+1, "fait la tâche ", t+1, " alors que cette tâche n'est pas dispo ", id+1)
                            b = False
                    elif s[t].x < tasks[L_tasks[T]].unavailabilities[id].unavailabilityEnd:
                        print("la tâche ", t+1, " est faite sur un créneau d'indisponibilité ", id+1)
                        b = False
if b :
    print("les indispos de tâches sont respectées")

# x[(number_of_tasks+u,r)]
# runav_id[u (general)]=(r,id (pour la personne))

les indispos de tâches sont respectées


In [ ]:
"""
- l’objectif est de maximiser la durée totale des tâches réalisées à coût opérationnel minimum ;
- chaque employé part en début de journée et rentre en fin de journée en un lieu qui lui est propre ;
- chaque employé possède une pause midi d’une durée d’1h à placer entre 12h et 14h, dans un lieu
quelconque ;
- certains employés peuvent avoir des périodes d’indisponibilité dans la journée, associées à des lieux
précisés. Les employés doivent se trouver en le lieu donné au début de la période et le quitter à la fin ;
- les lieux de certaines tâches peuvent être ouverts sur plusieurs créneaux disjoints dans la journée

- améliorer la data viz
"""